# Pull DUI Reports from UCONN Crash Repository for each town

This Jupyter notebook pulls DUI Report town data from the basic query tool of UCONN Crash Data Repository.

In [1]:
import pandas as pd
import requests
import json
import csv

In [2]:
# Disable InsecureRequestWarning warnings
# Taken from https://stackoverflow.com/questions/27981545/suppress-insecurerequestwarning-unverified-https-request-is-being-made-in-pytho
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Define a function to pull JSON data from an individual town

In [3]:
# Given the town ID, returns 
def get_town_report(town_id):
    
    base = 'https://www.ctcrash.uconn.edu/GetGrantReportResults.action'

    r = requests.post(base, params={
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-US,en;q=0.5',
            'Connection': 'keep-alive',
            'Content-Length': '304',
            'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'DNT': '1',
            'Host': 'www.ctcrash.uconn.edu',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:82.0) Gecko/20100101 Firefox/82.0'
        }, data={
            "formInput": '{"start":"2015","end":"2020","report":"grant_DUI","title":"","route_class":["-1"],"ru":["-1"],"location":{"scid":11,"val":"'
                + str(town_id)
                + '","mapid":null},"dataset":"CAS2MMUCC"}'
        }, verify=False)
    
    return json.loads( r.text )

## Fetch data for all towns
* Takes several minutes to complete

In [4]:
towns = pd.read_csv('id2town.csv')
towns['response'] = towns.id.apply(get_town_report)

### Generate three dataframes (crashes, fatalities, injuries)

In [5]:
def get_matrix(data, col):
    df = data.filter(['town', 'response'])
    df[col] = df.response.apply(lambda x: {year: count for year, count in x[col]})
    
    for year in range(2015, 2021):
        year = str(year)
        df[year] = df[col].apply(lambda x: int(x[year]) if year in x else 0)
        
    # Sum up to get CT towns by year
    ct = df.sum().rename('Connecticut')
    ct['town'] = 'Connecticut'
    
    return df.append(ct).filter(['town', '2015', '2016', '2017', '2018', '2019', '2020']).melt(id_vars='town', var_name='Year')


# Crashes
crashes = get_matrix(towns, 'totalCrashes')
crashes['Variable'] = 'DUI Crashes'

# Fatalities
fatalities = get_matrix(towns, 'fatalities')
fatalities['Variable'] = 'DUI Fatalities'

# Injuries
injuries = get_matrix(towns, 'injuries')
injuries['Variable'] = 'DUI Injuries'

/Users/ilyankou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  # Remove the CWD from sys.path while we load stuff.


### Combine dataframes into a single one

In [6]:
dui = pd.concat([ crashes, fatalities, injuries ]).sort_values(['town', 'Year'])
dui['Measure Type'] = 'Number'

### Add FIPS column

In [7]:
fips2town = pd.read_csv(
    'https://raw.githubusercontent.com/CT-Data-Collaborative/ct-town-county-fips-list/master/ct-town-county-fips-list.csv',
    usecols=['Town', 'FIPS'],
    dtype={'FIPS': str}
)

final = dui.merge(fips2town, how='left', left_on='town', right_on='Town')
final.FIPS = final.FIPS.fillna('09')

### Save

In [8]:
final.filter(['town', 'FIPS', 'Year', 'Measure Type', 'Variable', 'value']).rename(columns={
    'town': 'Town',
    'value': 'Value'
}).to_csv('data/dui-crashes-2015-2020.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)